In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os

In [ ]:
mask_height = 20 # в процентах потом от высоты изображения 1684
mask_widht = 45# в процентах потом от ширины изображения 1191

## Генератор из образцов маски

In [ ]:
main_path = "L:\\АвтоСтатистика\\Распознавание текста\\text_recognising\\russian_shrift\\"
main_path = "C:\\Users\\с\\Desktop\\Распознавание текста\\text_recognising\\russian_shrift\\"

In [ ]:
#путь к исходникам png
path_png = main_path + "DOCS_ALL\\ALL_PNG\\"#_correct\\"

In [ ]:
file_list = pd.DataFrame(columns=['file_name','mask','x_0','y_0'])

## Генерация образцов

In [ ]:
def generate(im_num, file_list, x_med, y_med):
    
    global main_path, path_png, mask_height, mask_widht
    
    #im_num=3
    image_file = file_list.loc[im_num,'file_name']
    image = np.array(Image.open(path_png+image_file))

    mask = file_list.loc[im_num,'mask']
    
    path_train = main_path + "DOCS_masks\\train\\" + mask + "\\"
    path_test  = main_path + "DOCS_masks\\test\\"  + mask + "\\"
    
    path_train_other = main_path + "DOCS_masks\\train\\other\\"
    path_test_other  = main_path + "DOCS_masks\\test\\other\\"

    mask = mask + '_b_' + str(im_num)
    
    x_0 = file_list.loc[im_num,'x_0']
    y_0 = file_list.loc[im_num,'y_0']


    #вправо вниз
    x_1 = x_0-1
    y_1 = y_0-2
    # вправо вверх
    x_2 = x_0+1
    # влево вверх
    y_2=y_0+2

    #trash frame
    x_f_1 = x_med - 2 * mask_height
    x_f_2 = x_med + 2 * mask_height
    y_f_1 = y_med - 3 * mask_widht
    y_f_2 = y_med + 3 * mask_widht

    #генерация масок
    i=0
    for x in range(x_1,x_2):
        for y in range(y_1,y_2):
            print(x,y)
            img = image[x:x+mask_height, y:y+mask_widht,:]
            fig = plt.figure(figsize=(8,4))
            plt.imshow(img) 
            plt.show()

            way = np.random.random_sample()
            if (way<0.03) or (way>0.97): 
                Image.fromarray(img).convert("RGB").save(path_test + mask + '_%s.png' % i)
                print(way, 'TEST!!!')
            else: 
                Image.fromarray(img).convert("RGB").save(path_train + mask + '_%s.png' % i)
                print(way, 'TRAIN!!!')
            i+=1

    #генерация мусора     
    i=0
    for x in range(x_f_1,x_f_2,6): #3
        for y in range(y_f_1,y_f_2,12):#6
            if (x not in range(x_1-2,x_2+2)) & (y not in range(y_1-5,y_2+5)):
                print(x,y)
                img = image[x:x+mask_height, y:y+mask_widht,:]
                fig = plt.figure(figsize=(8,4))
                plt.imshow(img) 
                plt.show()

                way = np.random.random_sample()
                if (way<0.001) or (way>0.99): 
                    Image.fromarray(img).convert("RGB").save(path_test_other + mask + '_oth_'+ str(im_num) + '_%s.png' % i)
                    print(way, 'TEST!!!')
                else: 
                    Image.fromarray(img).convert("RGB").save(path_train_other + mask + '_oth_'+ str(im_num) + '_%s.png' % i)
                    print(way, 'TRAIN!!!')
            i+=1 
    return print('ГОТОВО!!!')

In [ ]:
i=0
for filename in os.listdir(path_png): 
    #print(filename)
    file_list.loc[i,'file_name'] = filename
    i+=1

In [ ]:
file_list#.loc[73,:]

In [ ]:
im_num = 27
image_file = file_list.loc[im_num,'file_name']
image = np.array(Image.open(path_png+image_file))
im_shape = image.shape
print(im_shape)
fig = plt.figure(figsize=(24,16))
plt.imshow(image[int(im_shape[0]*0.05):int(im_shape[0]*0.20),:,:])
plt.show()


In [ ]:
mask = '05T'
x_0 = 191
y_0 = 768
img = image[x_0:x_0+mask_height, y_0:y_0+mask_widht,:]
fig = plt.figure(figsize=(6,4))
plt.imshow(img)
plt.show()
file_list.loc[im_num,'mask'] =  mask
file_list.loc[im_num,'x_0'] =  x_0
file_list.loc[im_num,'y_0'] =  y_0
#file_list.loc[im_num,:]

In [ ]:
path_csv = main_path + "DOCS_masks\\"
name = 'file_list_'+ mask + '.csv'
file_list.dropna(subset=['mask']).to_csv(path_csv + name)
#file_list.loc[im_num-2:im_num+2,:]

In [ ]:
#file_list = pd.read_csv(path_csv + 'file_list_01T.csv')
spisok  = file_list.dropna(subset=['mask'])
spisok

In [ ]:
x_med = int(spisok['x_0'].median())
y_med = int(spisok['y_0'].median())
print(x_med, y_med)

In [ ]:
x_med = spisok.groupby(['mask'])['x_0'].median().astype(int)[0]
y_med = spisok.groupby(['mask'])['y_0'].median().astype(int)[0]
print(x_med, y_med)

## Генерация

In [ ]:
for i in list(spisok.index): generate(i, spisok, x_med, y_med)
    #if i not in [0,1,22]: generate(i, spisok, x_med, y_med)

## THE END